Optimize and plot the gains/VSWR of a logperiodic antenna (6 brass elements, 75 Ohm transmission lines) for both the 2.4GHz as the 5.8GHz ISM bands.
    Inspired by an excercise for a course, hence the weird constraints.


In [1]:
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
import matplotlib as mpl

from PyNEC import *
from antenna_util import *

from context_clean import *

import math

brass_conductivity = 15600000 # mhos
copper_conductivity = 1.45e7 # Copper
ground_conductivity = 0.002
ground_dielectric = 10

tl_impedance = 75

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def n_seg(freq, length):
  wavelength = 3e8/(1e6*freq)
  return (2 * (int(math.ceil(77*length/wavelength))/2)) + 1

def sc_quad_helix(freq, height, diameter, wire_diameter = 0.02):
    
    nec = context_clean(nec_context())
    nec.set_extended_thin_wire_kernel(True)
    
    geo = geometry_clean(nec.get_geometry())

    wire_r = wire_diameter/2;
    helix_r = diameter/2;
    
    
    print "Wire Diameter %s" % (wire_r * 2)
    
    helix_turns = 0.5
    
    # helix loop 
    helix_twist_height = height / helix_turns
    geo.helix(tag_id=1, nr_segments=n_seg(freq,height), spacing=helix_twist_height, lenght=height, start_radius=np.array([helix_r, 0]), end_radius=np.array([helix_r, 0]), wire_radius=wire_r)
    geo.move(rotate_z=90, copies=3, tag_inc=1)
    geo.wire(tag_id=10, nr_segments=5, src=np.array([0, 0, height]), dst=np.array([helix_r, 0, height]), radius=wire_r)
    geo.wire(tag_id=11, nr_segments=5, src=np.array([0, 0, height]), dst=np.array([0, helix_r, height]), radius=wire_r)
    geo.wire(tag_id=12, nr_segments=5, src=np.array([0, 0, height]), dst=np.array([-helix_r, 0, height]), radius=wire_r)
    geo.wire(tag_id=13, nr_segments=5, src=np.array([0, 0, height]), dst=np.array([0, -helix_r, height]), radius=wire_r)
    
    # Everything is copper
    #nec.set_wire_conductivity(copper_conductivity)

    nec.geometry_complete(ground_plane=False)

    return nec

In [3]:
sc_quad_helix(143,1.5,0.5)

EK 0
Wire Diameter 0.02
GH 1 , 57 , 3.0 , 1.5 , 0.25 , 0.0 , 0.25 , 0.0 , 0.01
GM 1 , 3 , 0 , 0 , 90 , 0 , 0 , 0 , 0
GW 10 , 5 , 0.0 , 0.0 , 1.5 , 0.25 , 0.0 , 1.5 , 0.01
GW 11 , 5 , 0.0 , 0.0 , 1.5 , 0.0 , 0.25 , 1.5 , 0.01
GW 12 , 5 , 0.0 , 0.0 , 1.5 , -0.25 , 0.0 , 1.5 , 0.01
GW 13 , 5 , 0.0 , 0.0 , 1.5 , 0.0 , -0.25 , 1.5 , 0.01
GE 0


In [30]:
def geometry_logperiodic(l_1, x_1, tau):
  """
      x_1 is the distance from the origin to the largest (farthest away) dipole, which has a length of l_1.
      The spacing is as follows: l_{i+1}/l_i = tau = x_{i+1}/x_i
  """
  wire_radius = 0.00025 # 0.25 mm

  # alpha = np.arctan( (l_1/2.0) / x_1 )

  nec = context_clean(nec_context())
  nec.set_extended_thin_wire_kernel(True)

  geo = geometry_clean(nec.get_geometry())

  # Dipoles should be oriented in the Z direction; they should be placed on the (positive) X axis

  x_i = x_1
  l_i = x_1
  
  # As ususal, note that nec tags start at 1, and we typically index from 0!
  dipole_center_segs = {} # Maps from NEC wire id!
  
  dipoles_count = 5

  for dipole_tag in range(1, dipoles_count + 1):
      nr_segments = int(math.ceil(50*l_i/wavelength)) # TODO this might vary when sweeping even!
      #print nr_segments

      dipole_center_segs[dipole_tag] = nr_segments / 2 + 1

      center      = np.array([x_i, 0, 0])
      half_height = np.array([0  , 0, l_i/2.0])
      top         = center + half_height
      bottom      = center - half_height

      geo.wire(tag_id=dipole_tag, nr_segments=nr_segments, src=bottom, dst=top, radius=wire_radius)

      x_i = tau * x_i
      l_i = tau * l_i

  # Everything is in brass
  nec.set_wire_conductivity(brass_conductivity)

  nec.geometry_complete(ground_plane=False)

  # The 6th tag is the smallest tag is the source element
  for dipole in range(0, dipoles_count - 1):
      src_tag = 1 + dipole # NEC indexing
      src_seg = dipole_center_segs[src_tag]

      dst_tag = src_tag + 1
      dst_seg = dipole_center_segs[dst_tag]

      nec.transmission_line((src_tag, src_seg), (dst_tag, dst_seg), tl_impedance, crossed_line=True)

  smallest_dipole_tag = dipoles_count # Again, start at 1

  nec.voltage_excitation(wire_tag=smallest_dipole_tag, segment_nr=dipole_center_segs[smallest_dipole_tag], voltage=1.0)

  return nec

In [31]:
geometry_logperiodic(0.5, 0.5, 0.1)

In [32]:
start = 2300
stop  = 5900
count = stop - start


def get_gain_swr_range(l_1, x_1, tau, start=start, stop=stop, step=10):
    gains_db = []
    frequencies = []
    vswrs = []
    for freq in range(start, stop + 1, step):
        nec = geometry_logperiodic(l_1, x_1, tau)
        nec.set_frequency(freq) # TODO: ensure that we don't need to re-generate this!
        nec.radiation_pattern(thetas=Range(90, 90, count=1), phis=Range(180,180,count=1))

        rp = nec.context.get_radiation_pattern(0)
        ipt = nec.get_input_parameters(0)
        z = ipt.get_impedance()

        # Gains are in decibels
        gains_db.append(rp.get_gain()[0])
        vswrs.append(vswr(z, system_impedance))
        frequencies.append(ipt.get_frequency())

    return frequencies, gains_db, vswrs

def create_optimization_target():
  def target(args):
      l_1, x_1, tau = args
      if l_1 <= 0 or x_1 <= 0 or tau <= 0:
          return float('inf')

      try:
        result = 0

        vswr_score = 0
        gains_score = 0

        for range_low, range_high in [ (2400, 2500), (5725, 5875) ]:
            freqs, gains, vswrs = get_gain_swr_range(l_1, x_1, tau, start=range_low, stop=range_high)

            for gain in gains:
                gains_score += gain
            for vswr in vswrs:
                if vswr >= 1.8:
                    vswr = np.exp(vswr) # a penalty :)
                vswr_score += vswr

        # VSWR should minimal in both bands, gains maximal:
        result = vswr_score - gains_score

      except:
          print "Caught exception"
          return float('inf')

      print result

      return result
  return target


def simulate_and_get_impedance(nec):
  nec.set_frequency(design_freq_mhz)

  nec.xq_card(0)

  index = 0
  return nec.get_input_parameters(index).get_impedance()

system_impedance = 50 # This makes it a bit harder to optimize, given the 75 Ohm TLs, which is good for this excercise of course...

# (2.4 GHz to 2.5 GHz) and the 5.8 GHz ISM band (5.725 GHz to 5.875 GHz)

design_freq_mhz = 2450 # The center of the first range
wavelength = 299792e3/(design_freq_mhz*1000000)

majorLocator = mpl.ticker.MultipleLocator(10)
majorFormatter = mpl.ticker.FormatStrFormatter('%d')
minorLocator = mpl.ticker.MultipleLocator(1)
minorFormatter = mpl.ticker.FormatStrFormatter('%d')

def draw_frequencie_ranges(ax):
    ax.axvline(x=2400, color='red', linewidth=1)
    ax.axvline(x=2500, color='red', linewidth=1)
    ax.axvline(x=5725, color='red', linewidth=1)
    ax.axvline(x=5875, color='red', linewidth=1)

def show_report(l1, x1, tau):
    nec = geometry_logperiodic(l1, x1, tau)

    z = simulate_and_get_impedance(nec)

    print "Initial impedance: (%6.1f,%+6.1fI) Ohms" % (z.real, z.imag)
    print "VSWR @ 50 Ohm is %6.6f" % vswr(z, 50)

    nec = geometry_logperiodic(l1, x1, tau)
  
    freqs, gains, vswrs = get_gain_swr_range(l1, x1, tau, step=5)

    freqs = np.array(freqs) / 1000000 # In MHz
  
    ax = plt.subplot(111)
    ax.plot(freqs, gains)
    draw_frequencie_ranges(ax)

    ax.set_title("Gains of a 5-element log-periodic antenna")
    ax.set_xlabel("Frequency (MHz)")
    ax.set_ylabel("Gain")

    ax.yaxis.set_major_locator(majorLocator)
    ax.yaxis.set_major_formatter(majorFormatter)

    ax.yaxis.set_minor_locator(minorLocator)
    ax.yaxis.set_minor_formatter(minorFormatter)

    ax.yaxis.grid(b=True, which='minor', color='0.75', linestyle='-')

    plt.show()

    ax = plt.subplot(111)
    ax.plot(freqs, vswrs)
    draw_frequencie_ranges(ax)

    ax.set_yscale("log")
    ax.set_title("VSWR of a 6-element log-periodic antenna @ 50 Ohm impedance")
    ax.set_xlabel("Frequency (MHz)")
    ax.set_ylabel("VSWR")

    ax.yaxis.set_major_locator(majorLocator)
    ax.yaxis.set_major_formatter(majorFormatter)
    ax.yaxis.set_minor_locator(minorLocator)
    ax.yaxis.set_minor_formatter(minorFormatter)
  
    ax.yaxis.grid(b=True, which='minor', color='0.75', linestyle='-')
    plt.show()

In [3]:
  initial_l1  = wavelength / 2
  initial_x1  = wavelength / 2
  initial_tau = 0.8

  print "Wavelength is %0.4fm, initial length is %0.4fm" % (wavelength, initial_l1)
  
  print "Unoptimized antenna..."
  show_report(initial_l1, initial_x1, initial_tau)

  print "Optimizing antenna..."
  target = create_optimization_target()

  # Optimize local minimum only with gradient desce
  #optimized_result = scipy.optimize.minimize(target, np.array([initial_l1, initial_x1, initial_tau]), method='Nelder-Mead')

  # Use differential evolution:
  minimizer_kwargs = dict(method='Nelder-Mead')
  bounds = [ (0.01, 0.2), (0.01, 0.2), (0.7, 0.9) ]
  optimized_result = scipy.optimize.differential_evolution(target, bounds, seed=42, disp=True, popsize=20)

  # Basin hopping isn't so good, but could also have been an option:
  #optimized_result = scipy.optimize.basinhopping(target, np.array([initial_l1, initial_x1, initial_tau]), minimizer_kwargs=minimizer_kwargs, niter=5, stepsize=0.015, T=2.0, disp=True)

  print "Optimized antenna..."
  optimized_l1, optimized_x1, optimized_tau =  optimized_result.x[0], optimized_result.x[1], optimized_result.x[2]
  show_report(optimized_l1, optimized_x1, optimized_tau)


Wavelength is 0.1224m, initial length is 0.0612m
Unoptimized antenna...
Initial impedance: (  68.7, +15.2I) Ohms
VSWR @ 50 Ohm is 1.504232
Optimizing antenna...
[ 170067.57691283]
[  2.99352006e+10]
[ 664.2220855]
[ 3172.92141886]
[  2.21130758e+14]
[  1.65932765e+14]
[-72.51254898]
[  7.63701399e+162]
[-85.88192809]
[ inf]
[ 162.31141884]
[ 2814657.84820774]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:122: RuntimeWarning: overflow encountered in exp


[ 56219791.79510481]
[ 18389.94075227]
[ 353.62636522]
[  1.04932153e+10]
[  1.72020809e+63]
[ 337.17978192]
[ 36.15144309]
[ inf]
[ 1214.97294178]
[ 77.61071636]
[ 1173.85291273]
[ 38.56281545]
[ 11137.11554739]
[ 227.97267165]
[  1.42045473e+78]
[  1.80043447e+08]
[ inf]
[  6.90097778e+299]
[ 3767.86482674]
[ 8732.28863228]
[ 617.89070673]
[ 4454.19322561]
[ 800.37870254]
[-27.46209332]
[ 176.58148057]
[ inf]
[ 7.93680992]
[ inf]
[ 9989.73170367]
[ 23111.85741754]
[ 26.86474651]
[ 266.00280082]
[-67.95101194]
[ 149.06319162]
[ 471.85372324]
[  4.76263890e+08]
[  6.91955508e+99]
[ 23002.38276151]
[ 856.38344357]
[ 27502.95254484]
[ 172.33542013]
[ 468275.74625198]
[ 186.11469588]
[ 379.27366099]
[-114.46329135]
[ 170.41566677]
[ 14602.1149273]
[  1.45178200e+08]
[ 11515.15454498]
[ 146.90249581]
[ 24557.27793248]
[ inf]
[ 4474.43904716]
[-65.3828824]
[-72.51254898]
[  1.15124135e+133]
[ 62585.72296157]
[  1.18280525e+14]
[ 58597.07656705]
[  5.38593198e+08]
[ 56219791.79510481]
[ inf]

/usr/lib/python2.7/dist-packages/numpy/core/_methods.py:101: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


differential_evolution step 1: f(x)= -114.463
[ 618740.32197909]
[ 550182.96070474]
[ 1869.54312649]
[ 214.00305106]
[ 381.44324215]
[-66.43497009]
[ 321.3269855]
[ 930.8959931]
[ inf]
[  1.18280525e+14]
[ inf]
[ 42584704.23346418]
[  1.49188163e+64]
[-94.0676438]
[  5.72728318e+09]
[ 808.27810112]
[  2.27621864e+85]
[ 96.81384044]
[-9.15306632]
[  2.12095806e+10]
[ 162.94634782]
[ 390.33215525]
[ 300.64266903]
[  7.80311963e+17]
[ 16363418.44605458]
[ 87.0975673]
[ 12969271.2076769]
[  1.70859760e+63]
[-91.34584279]
[ 8065318.63810766]
[  2.23900035e+31]
[ 12044.72045611]
[  1.76096267e+17]
[ 1088.12148031]
[ 245.31172266]
[ 66.08423299]
[ 1.24334922]
[ inf]
[  3.05877450e+112]
[  1.96921542e+15]
[ 5269.0413253]
[ 82.702122]
[ 177.02112642]
[ 464.66818866]
[ 9379.73268852]
[ 193.81147953]
[ inf]
[  1.51429799e+08]
[ 1664.2838717]
[ 159.29596769]
[ 338.21607614]
[ 4892852.38468401]
[ 193.19507344]
[ 913.01764471]
[  6.39686773e+19]
[-23.86098566]
[ 170067.57691283]
[ 27591.70735803]
[-